## Finetune Llama 2 7B Chat

In this Google Colab notebook, we will fine-tune Meta's [Llama 2 7b Chat Huggingface](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) model.

The Colab T4 GPU has 16 GB of VRAM, which is barely enough to store Llama 2 7B's weights, which means full fine-tuning is not possible, hence we use Parameter-Efficient-Fine-Tuning [PEFT]() techniques such as [LoRA]() & [QLoRA]().

QLoRA technique is memory efficient finetuning that combines [quantization]() and [LoRA](). We fine tune the model in 4-bit precision to optimize VRAM usage. We will load the large model in 4-bit using `bitsandbytes`.

For fine-tuning, we are going to rely on these HF & other libraries:
- `transformers`: A library to facilitate to download and use pre-trained models.

- `datasets`: Facilitates loading datasets.

- `accelerate`: Library to enhance the inference speed of the model.

- `peft`: Parameter-Efficient Fine-Tuning (PEFT) is a library for efficiently fine-tuning LLMs without touching all of the LLM's parameters. PEFT supports QLoRA method to fine-tune a small fraction of LLM parameters with 4-bit quantization.

- `trl`: Transformer Reinforcement Learning (TRL), Supervised Fine-Tuning Trainer API (SFTTrainer) makes it easy to train models on custom datasets.

- `bitsandbytes`: to quantize LLM into 4-bit or 8-bit.
- `wandb`: A tool that serves as monitoring platform to track out training metrics.

### Prerequisites

Before proceed further, you must meet below pre-requisites,
- Hugging face account to download pre-trained model weights and datasets.
- Signup for Weights & Biases and obtain an API key

### Setup

Run the cell below to install required libraries.

In [ ]:
!pip install -q -U torch transformers datasets accelerate peft trl bitsandbytes wandb

Load necessary modules from above libraries.

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
# Model and tokenizer names
model_name = "meta-llama/Llama-2-7b-chat-hf"

fine_tuned_model_name = "llama-2-7b-chat-hf-enhanced"

# Hugging face repository link to save fine-tuned model
# Create new repository in huggingface, copy and paste here

### Dataset

For finetuning to specific task, we will use Guanaco dataset.

The dataset can be found [here]()

In [ ]:
# Dataset
data_name = "mlabonne/guanaco-llama2-1k"
training_data = load_dataset(data_name, split="train")
training_data['text'][0]

#### Log into hugging face hub

In [ ]:
notebook_login()

### Quantization Configuration

QLoRA quantizes a pre-trained language model to 4 bits and freezes the parameters. Later, a small number of trainable Low-Rank Adapter layers are then added to model.

`Trivia`:
- What is [mixed precision training](https://huggingface.co/docs/transformers/main/en/perf_train_gpu_one#mixed-precision-training)?
- Which is best quantization scheme among bitsandbytes, AutoGPTQ, Activation-aware Weight Quantization (AWQ)?

Create 4-bit quantization with NF4 type configuration using BitsAndBytes.

In [ ]:
# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

### Loading the model

In this section we will load the Llama 2 7B Chat model, quantiae it in 4bit and attach LoRA adapters on it.

In [ ]:
# Model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

### Tokenization

Set up the tokenizer.

Add padding as it makes training use less memory.

In [ ]:
# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.add_eos_token = True
llama_tokenizer.add_bos_token, llama_tokenizer.add_eos_token
llama_tokenizer.padding_side = "right"

### LoRA Configuration

In [ ]:
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj"
    ]
)

### Training Arguments

In [ ]:


# Training Params
train_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    optim="paged_adamw_8bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="linear",
    report_to="wandb"
)

### Load the Trainer

We use SFTTrainer from TRL library that gives wrapper around transformers `Trainer` to easily fine-tune models on instruction based dataset using PEFT adapters.

Let us load the training arguments below.

In [ ]:
# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=training_data,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params
)

#### Monitoring

In [ ]:
# Login to Weights & Baises
wandb.login(key="Replace with your Authorization token here")
run = wandb.init(project='Fine-tune Llama 2 7B Chat', job_type='training', anonymous='allow')

### Train the model

# Training
trainer.train()

# Save Model
trainer.model.save_pretrained(fine_tuned_model_name)

In [ ]:
wandb.finish()

In [ ]:
model.config.use_cache = True
model.eval()

### Inference Fine tuned Model

In [ ]:
# Generate Text
query = "How do I use the OpenAI API?"
text_gen = pipeline(task="text-generation", model=refined_model, tokenizer=llama_tokenizer, max_length=200)
output = text_gen(f"<s>[INST] {query} [/INST]")
print(output[0]['generated_text'])

In [ ]:
# Clear memory footprint
del model, trainer
torch.cuda.empty_cache()

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"":0}
)
model = PeftModel.from_pretrained(base_model, fine_tuned_model_name)
model = model.merge_and_uload()


# Reload tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
model.push_to_hub(fine_tuned_model_name)
tokenizer.push_to_hub(fine_tuned_model_name)